# For each localization grid, randomly select Ovenbird localized singing events to manually review

> Note: the full audio data used by this script is not provided in this repository, the script is provided for reference only


In [ ]:
import numpy as np
import pandas as pd
from glob import glob
import random

np.random.seed(2024)
random.seed(2024)

In [ ]:
filtered_events_dir = "REDACTED"  # matches out_dir of 5_remove_duplicate_events.py
splits_path = "REDACTED"  # location to save val/test set csvs, matches splits_path of 7_annotate_localized_songs.ipynb

In [ ]:
# count localized events
random.seed(2024)

clip_dfs = pd.concat([pd.read_csv(p) for p in glob(f"{filtered_events_dir}/*.csv")])
clip_dfs.array.value_counts().to_csv("n_localized_events_per_array.csv")
print(len(clip_dfs))
clip_dfs.array.value_counts()

60961


array
SBT-3-51    12084
SBT-3-7     10004
SBT-3-49     9786
SBT-6-76     7565
SBT-6-79     6860
SBT-3-9      4116
SBT-3-15     3066
SBT-3-33     2726
SBT-3-32     1887
SBT-3-18     1378
SBT-6-81      813
SBT-3-39      507
SBT-6-83      169
Name: count, dtype: int64

In [ ]:
random.seed(2024)

clip_dfs = pd.concat(
    [pd.read_csv(p) for p in glob(f"{filtered_events_dir}/*.csv")]
).reset_index(drop=True)


# split into train, val, and test
# SBT-6-83 only has a handful of Ovenbird detections; 3-7 has no clear territories from localization; 6-76 has 3 clear territories;
test_df = clip_dfs[
    clip_dfs.array.apply(lambda x: x in ("SBT-6-83", "SBT-3-7", "SBT-6-76"))
]
trainval = clip_dfs.drop(test_df.index).reset_index(drop=True)
test_df = test_df.reset_index()
# test_df.to_csv(f"{splits_path}/test_unlabeled.csv", index=False)

# select 100 events from each array, annotate closest, then apply label to all clips from the array
N_events_to_annotate_test = 100
events_to_annotate = []
for array, df_subset in test_df.groupby("array"):
    event_ids = sorted(df_subset.event_id.unique())
    if len(event_ids) > N_events_to_annotate_test:
        event_ids = random.sample(event_ids, N_events_to_annotate_test)
    events_to_annotate.extend(event_ids)

test_to_annnotate = test_df[test_df.event_id.apply(lambda x: x in events_to_annotate)]

test_to_annnotate.to_csv(f"{splits_path}/test_to_annotate.csv", index=False)
# only annotate the closest clip

In [ ]:
# from the validaton and test set data, select stratified random subset of _events_ to annotate
# always annotate the nearest clip;
# then assign labels to other clips from the same event
# these will form the validation set
random.seed(2024)

N_events_to_annotate = 50
events_to_annotate = []
for array, df_subset in trainval.groupby("array"):
    event_ids = df_subset.event_id.unique()
    if len(event_ids) > N_events_to_annotate:
        event_ids = random.sample(sorted(event_ids), N_events_to_annotate)
    events_to_annotate.extend(event_ids)

val_df = trainval[trainval.event_id.apply(lambda x: x in events_to_annotate)]
train_df = trainval.drop(val_df.index)

[len(df.event_id.unique()) for df in (train_df, val_df, test_df, test_to_annnotate)]

[7200, 500, 3228, 233]

In [ ]:
# val_df.to_csv(f"{splits_path}/test_set_to_anotate.csv", index=False)
# train_df.to_csv(f"{splits_path}/validation_set_to_annotate_df.csv", index=False)

Next:
1. annotate individual identities for the closest clip in each event for val_df and test_df_to_annotate
2. apply ID labels to other clips from the same event
3. compare labels to spatial positions
4. use as evaluation for AIID feature extractors
